In [3]:
import win32api
import win32con

# ShellExecute parameters:
# hwnd (0), operation ("open"), file (executable or shell URI), parameters, directory, show command
win32api.ShellExecute(
    0,
    "open",
    "explorer.exe",
    "shell:Appsfolder\\Microsoft.MicrosoftSolitaireCollection_8wekyb3d8bbwe!App",
    None,
    win32con.SW_SHOWNORMAL
)


42

In [ ]:
import cv2
import time
from ultralytics import YOLO

emotion_classes = ['Angry','Boring','Disgust','Fear','Happy','Neutral','Sad','Stress','Suprise']
model = YOLO("D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/yolo11n-cls.pt")

cap = cv2.VideoCapture(0)
inference_interval = 5  # seconds
last_inference_time = 0
last_label = "No emotion detected"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    if current_time - last_inference_time >= inference_interval:
        results = model(frame, conf=0.1)  # lower confidence threshold
        last_inference_time = current_time

        if results[0].boxes is not None and len(results[0].boxes) > 0:
            for box in results[0].boxes:
                class_id = int(box.cls[0])
                confidence = box.conf[0].item()
                label = f"{emotion_classes[class_id]} ({confidence:.2f})"
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            last_label = label
        else:
            last_label = "No emotion detected"

    cv2.putText(frame, last_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("YOLOv11 Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
from ultralytics import YOLO

# Load Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load your YOLO classification model
model = YOLO("D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/48x48_yolo11.pt")

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)  # (x, y, w, h) for each face

    for (x, y, w, h) in faces:
        # Crop the face from the frame
        face_img = frame[y:y+h, x:x+w]

        # Convert to grayscale
        gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)

        # Resize to 48x48
        resized_face = cv2.resize(gray_face, (48, 48))

        # Convert back to 3-channel if your YOLO model expects RGB input
        input_face = cv2.cvtColor(resized_face, cv2.COLOR_GRAY2BGR)

        # Run YOLO classification on the preprocessed face
        results = model(input_face)

        # Visualize results (optional: you can draw on the original frame)
        annotated_face = results[0].plot()

        # Resize the annotated face back to original face size
        frame[y:y+h, x:x+w] = cv2.resize(annotated_face, (w, h))

        # Draw bounding box around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show the frame with annotations
    cv2.imshow("Face Detection + YOLO Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 64x64 disgust 0.36, sad 0.18, fear 0.13, stress 0.13, surprise 0.10, 11.5ms
Speed: 13.9ms preprocess, 11.5ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 sad 0.57, disgust 0.25, neutral 0.09, stress 0.06, boring 0.01, 10.3ms
Speed: 2.9ms preprocess, 10.3ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 sad 0.39, disgust 0.31, neutral 0.19, stress 0.06, angry 0.02, 14.2ms
Speed: 2.2ms preprocess, 14.2ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 neutral 0.43, sad 0.33, disgust 0.13, stress 0.04, boring 0.03, 8.8ms
Speed: 1.8ms preprocess, 8.8ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 disgust 0.64, neutral 0.20, sad 0.08, boring 0.03, surprise 0.02, 13.0ms
Speed: 2.6ms preprocess, 13.0ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 sad 0.52, disgust 0.26, neutral 0.15, stress 0.04, boring 0.02, 10.1ms
Speed: 1.5ms preprocess, 10.1ms inferenc

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

# Load Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load YOLO classification model
model = YOLO("D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/48x48_yolo11.pt")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    top5_display = []  # To store top 5 predictions for display

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        resized_face = cv2.resize(gray_face, (48, 48))
        input_face = cv2.cvtColor(resized_face, cv2.COLOR_GRAY2BGR)

        results = model(input_face)
        pred = results[0]

        # Extract and sort top 5 predictions
        if pred.probs is not None:
            probs = pred.probs.data.cpu().numpy()
            top5_indices = np.argsort(probs)[-5:][::-1]  # Top 5 in descending order
            for idx in top5_indices:
                class_name = model.names[idx]
                confidence = probs[idx]
                top5_display.append(f"{class_name}: {confidence:.2f}")

        annotated_face = pred.plot()
        frame[y:y+h, x:x+w] = cv2.resize(annotated_face, (w, h))

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        break  # Optional: only classify and display for the first face

    # Draw top-5 results on top-left corner
    y_offset = 30
    for line in top5_display:
        cv2.putText(frame, line, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 0, 0), 2, cv2.LINE_AA)
        y_offset += 30

    cv2.imshow("Face Detection + YOLO Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 64x64 stress 0.96, disgust 0.01, neutral 0.01, happy 0.01, surprise 0.01, 11.6ms
Speed: 4.2ms preprocess, 11.6ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 neutral 0.34, happy 0.26, disgust 0.13, sad 0.10, stress 0.09, 9.5ms
Speed: 2.5ms preprocess, 9.5ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 neutral 0.39, sad 0.31, stress 0.19, disgust 0.04, fear 0.02, 8.8ms
Speed: 1.6ms preprocess, 8.8ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 neutral 0.47, stress 0.15, happy 0.09, disgust 0.09, sad 0.07, 8.7ms
Speed: 1.8ms preprocess, 8.7ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 boring 0.62, surprise 0.13, neutral 0.11, disgust 0.08, sad 0.03, 20.2ms
Speed: 1.9ms preprocess, 20.2ms inference, 0.1ms postprocess per image at shape (1, 3, 64, 64)

0: 64x64 sad 0.32, disgust 0.25, stress 0.13, neutral 0.08, surprise 0.08, 9.6ms
Speed: 1.9ms preprocess, 9.6ms inference, 